In [1]:
pip install transformers faiss-cpu


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer, pipeline
import json
import re

context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

file_path = "/home/abhay/my_projects/ps2_chatbot/life_at_jklu/health_center.txt"

with open(file_path, 'r', encoding='utf-8') as f:
    uploaded_text = f.read()

question_generator = pipeline('text2text-generation', model='valhalla/t5-small-qg-prepend')

def generate_questions(text):
    generated_questions = question_generator(f"generate question: {text}")
    return generated_questions

def extract_answer(question, context):
    key_terms = re.split(r"\s|,", question.lower())
    for sentence in context.split(". "):
        if all(term in sentence.lower() for term in key_terms if term):
            return sentence
    return "Answer not found"

qa_pairs = []
questions = generate_questions(uploaded_text)

for q in questions:
    question_text = q['generated_text']
    answer_text = extract_answer(question_text, uploaded_text)
    qa_pairs.append({
        "question": question_text,
        "answer": answer_text
    })

output_file = "qa_pairs_single_file.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(qa_pairs, f, ensure_ascii=False, indent=4)

print(f"Question-answer pairs saved to {output_file}")
